In [10]:
#!/usr/bin/python

import sys
import math
from gurobipy import *
import numpy

#  Solve min  (3 sin(x1) - 2 sin(x2))**2 + 10 x_3^4
#        s.t.  x1 + x2 + x3 >= 1
#              5x1 + 4x2 >= 2
#               x1, x2, x3 >= 0
#

def linesearch(n, x, delta):
    for h in range(100):
        value = myfunction(x + (h/100.)*delta)
def myfunction(x):
    return (3*math.sin(x[0]) - 2*math.sin(x[1]))**2 + 10*x[2]*x[2]*x[2]*x[2]
def loop(n,xsol,row1,row2):
    x = numpy.zeros(n)
    gradient = numpy.zeros(n)
    slacks = numpy.zeros(2)
    for j in range(n):
        x[j] = xsol[j]

    maxits = 1
    for itcount in range(maxits):
        val = myfunction(x)
        gradcomp(n, gradient, x)
        print ("hello, value = ", val)
        compute_slacks(n, row1, row2, x, slacks)
        steplp(n, row1, row2, x, slacks, gradient)

def compute_slacks(n, row1, row2, x, slacks):
    #first handle row1
    sum = 0
    for j in range(n):
        sum += row1[j]*x[j]
    slacks[0] = sum - 1.0
    sum = 0
    for j in range(n):
        sum += row2[j]*x[j]
    slacks[1] = sum - 2.0

    print ("slacks is:", slacks)


def gradcomp (n, gradient, x):
    Delta = 2*(3*math.sin(x[0])- 2*math.sin(x[1]))
    gradient[0] = Delta*3*math.cos(x[0])
    gradient[1] = Delta*(-2)*math.cos(x[1])
    gradient[2] = 40*(x[2])*(x[2])*(x[2])

    #print gradient

def steplp(n, row1, row2, x, slacks, gradient):
    m = Model("feasible")
    m.ModelSense = 1 #minimize
    #create variables and put into a dictionary
    deltavar = {}
    for j in range(n):
        deltavar[j] = m.addVar(lb = -1.0, ub = 1.0, obj = gradient[j], vtype = GRB.CONTINUOUS, name = "delta_"+str(j+1))

    # Update model to integrate new variables
    m.update()    
    #add first constraint
    xpr = LinExpr()
    for j in range(n):
        xpr += row1[j]*deltavar[j]
    m.addConstr(xpr >= -slacks[0], name="const1")        
    xpr = LinExpr()
    for j in range(n):
        xpr += row2[j]*deltavar[j]
    m.addConstr(xpr >= -slacks[1], name="const2")        
    m.update()
    m.write("step.lp")
    m.optimize()

    code = "optimal"

    if m.status == GRB.status.INF_OR_UNBD:
        print('->LP infeasible or unbounded\n')
        code = "inf_or_unbd"
    if m.status == GRB.status.UNBOUNDED:
        print('->LP unbounded\n')
        code = "unbd"
    if m.status == GRB.status.INFEASIBLE:
        print('->LP infeasible\n')
        code = "infeas"


    print(" -> LP optimal, value = %g\n" %(m.objval))

    deltasol = numpy.zeros(n)
    for j in range(n):
        print ("%s = %g" %(deltavar[j].varname, deltavar[j].x))
        deltasol[j] = deltavar[j].x

    print (deltasol)

def feasible(n, row1, row2):
    m = Model("feasible")
    m.ModelSense = 1 #minimize
    #create variables and put into a dictionary
    xvar = {}
    for j in range(n):
        xvar[j] = m.addVar(lb = 0, ub = GRB.INFINITY, obj = 1, vtype = GRB.CONTINUOUS, name = "x_"+str(j+1))

    # Update model to integrate new variables
    m.update()

    #add first constraint
    xpr = LinExpr()
    for j in range(n):
        xpr += row1[j]*xvar[j]
    m.addConstr(xpr >= 1, name="const1")        
    xpr = LinExpr()
    for j in range(n):
        xpr += row2[j]*xvar[j]
    m.addConstr(xpr >= 2, name="const2")

    m.update()
    m.write("feas.lp")

    m.optimize()

    code = "optimal"

    if m.status == GRB.status.INF_OR_UNBD:
        print('->LP infeasible or unbounded\n')
        code = "inf_or_unbd"
    if m.status == GRB.status.UNBOUNDED:
        print('->LP unbounded\n')
        code = "unbd"
    if m.status == GRB.status.INFEASIBLE:
        print('->LP infeasible\n')
        code = "infeas"


    print(" -> LP optimal, value = %g\n" %(m.objval))

    xsol = numpy.zeros(n)
    for j in range(n):
        print ("%s = %g" %(xvar[j].varname, xvar[j].x))
        xsol[j] = xvar[j].x

    return code, xsol
        



def run():
    n = 3
    #0 Create arrays for the two constraints
    row1 = numpy.zeros(n, dtype='double')
    row1[0] = row1[1] = row1[2] = 1.0
    row2 = numpy.zeros(n, dtype='double')
    row2[0] = 5.0
    row2[1] = 4.0
    #if we know constraints are sparse, store them using numpy or scipy sparse
    #structures
#     print "row1 = ",
    print ("row1 = ", row1)
#     print "row2 = ",
    print ("row2 = ",row2)

    code, xsol = feasible(n,row1,row2)

#     print "initial feasible solution = ",
    print("initial feasible solution = ", xsol)

    loop(n,xsol,row1,row2)

if __name__ == '__main__':
    run()




row1 =  [1. 1. 1.]
row2 =  [5. 4. 0.]
Optimize a model with 2 rows, 3 columns and 5 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+00]
Presolve time: 0.01s
Presolved: 2 rows, 3 columns, 5 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.250000e+00   0.000000e+00      0s
       2    1.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds
Optimal objective  1.000000000e+00
 -> LP optimal, value = 1

x_1 = 0.4
x_2 = 0
x_3 = 0.6
initial feasible solution =  [0.4 0.  0.6]
hello, value =  2.6608198079377554
slacks is: [0. 0.]
Optimize a model with 2 rows, 3 columns and 5 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [5e+00, 9e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 1 rows and 1 columns
Presolve ti